## Test notebook for the MegaDetector Flask API

### Imports and environment

In [2]:
import requests
import os
import json
import io

from requests_toolbelt import MultipartEncoder
from requests_toolbelt.multipart import decoder
from PIL import Image

# sample_input_dir should point to a folder full of png/jpg files
sample_input_root = os.getcwd()
sample_input_root = sample_input_root.replace('\\api_flask_redis\\api_core\\tests', '\\api\\synchronous\\sample_input')
sample_input_dir = os.path.join(sample_input_root, 'png')

ip_address = 'onemoremallard.southcentralus.cloudapp.azure.com'
port = '5050'
detect_endpoint = 'http://{}:{}/v1/camera-trap/sync/detect'.format(ip_address, port)
print(detect_endpoint)

http://onemoremallard.southcentralus.cloudapp.azure.com:5050/v1/camera-trap/sync/detect


### Select images to submit

In [5]:
filenames = os.listdir(sample_input_dir)
filenames = [s for s in filenames if (s.lower().endswith('.png') or s.lower().endswith('.jpg'))]
filenames = [os.path.join(sample_input_dir,s) for s in filenames]
    
print('Found {} images files:'.format(len(files)))
for fn in filenames:
    print(fn)

Found 4 images files:
c:\git\cameratraps\api\synchronous\sample_input\png\S1_D04_R6_PICT0022.JPG
c:\git\cameratraps\api\synchronous\sample_input\png\S1_D04_R6_PICT0022.png
c:\git\cameratraps\api\synchronous\sample_input\png\S1_D04_R6_PICT0128.JPG
c:\git\cameratraps\api\synchronous\sample_input\png\S1_D04_R6_PICT0128.png


### Submit images

In [16]:
params = {
    'min_confidence': 0.15,
    'min_rendering_confidence': 0.8,
    'render': True,
    'key': None
}

def clean_filename(s):
    s = s.replace('/','_').replace('\\','_')
    return s
    
file_handles = {}
for fn in filenames:
    file_handles[fn] = (clean_filename(fn), open(fn, 'rb'), 'image/jpeg')

m = MultipartEncoder(fields=files)
print(m.content_type)

r = requests.post(detect_endpoint, 
                  params=params,
                  data=m,
                  headers={'Content-Type': m.content_type})

print('Status: {}'.format(r.status_code))

if not r.ok:
    print('Error: {}\n{}'.format(r.reason,r.text))
    
print('Elapsed time: {}'.format(r.elapsed.total_seconds()))

for f in file_handles.values():
    f[1].close()

multipart/form-data; boundary=8fa571a65d5d42da9bcadc2d9e9bbf11


### Decode and display results

In [12]:
results = decoder.MultipartDecoder.from_response(r)
text_results = {}
images = {}
for part in results.parts:
    # part is a BodyPart object with b'Content-Type', and b'Content-Disposition', the later includes 'name' and 'filename' info
    headers = {}
    for k, v in part.headers.items():
        headers[k.decode(part.encoding)] = v.decode(part.encoding)
    if headers.get('Content-Type', None) == 'image/jpeg':
        #images[part.headers['filename']] = part.content
        c = headers.get('Content-Disposition')
        image_name = c.split('name="')[1].split('"')[0]  # somehow all the filename and name info is all in one string with no obvious forma
        image = Image.open(io.BytesIO(part.content))
        
        images[image_name] = image
    
    elif headers.get('Content-Type', None) == 'application/json':
        text_result = json.loads(part.content.decode())
text_result

for img_name, img in sorted(images.items()):
    print(img_name)
    img
    print()
    